In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc, col, max, struct, date_format

import matplotlib.pyplot as plt

In [2]:
spark = SparkSession.builder.appName("spark_app").getOrCreate()

In [3]:
questions_csv_path = "../dataset/questions.csv"
questions_df = spark.read.format("csv").option("inferSchema", True).option("header", True).load(questions_csv_path)

In [12]:
questions_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- ClosedDate: string (nullable = true)
 |-- DeletionDate: string (nullable = true)
 |-- Score: integer (nullable = true)
 |-- OwnerUserId: string (nullable = true)
 |-- AnswerCount: string (nullable = true)



In [14]:
(questions_df.count(), len(questions_df.columns))

(17203824, 7)

In [4]:
questions_df.show()

+---+-------------------+--------------------+--------------------+-----+-----------+-----------+
| Id|       CreationDate|          ClosedDate|        DeletionDate|Score|OwnerUserId|AnswerCount|
+---+-------------------+--------------------+--------------------+-----+-----------+-----------+
|  1|2008-08-01 04:26:37|                  NA|2011-03-28T00:53:47Z|    1|         NA|          0|
|  4|2008-08-01 04:42:52|                  NA|                  NA|  458|          8|         13|
|  6|2008-08-01 05:08:08|                  NA|                  NA|  207|          9|          5|
|  8|2008-08-01 06:33:19|2013-06-03T04:00:25Z|2015-02-11T08:26:40Z|   42|         NA|          8|
|  9|2008-08-01 06:40:59|                  NA|                  NA| 1410|          1|         58|
| 11|2008-08-01 06:55:37|                  NA|                  NA| 1129|          1|         33|
| 13|2008-08-01 07:42:38|                  NA|                  NA|  451|          9|         25|
| 14|2008-08-01 07:5

In [6]:
questions_tags_csv_path = "../dataset/questions_tags.csv"
questions_tags_df = spark.read.format("csv").option("inferSchema", True).option("header", True).load(questions_tags_csv_path)

In [9]:
questions_tags_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Tag: string (nullable = true)



In [15]:
(questions_tags_df.count(), len(questions_tags_df.columns))

(50576842, 2)

In [7]:
questions_tags_df.show()

+---+-------------------+
| Id|                Tag|
+---+-------------------+
|  1|               data|
|  4|                 c#|
|  4|           winforms|
|  4|    type-conversion|
|  4|            decimal|
|  4|            opacity|
|  6|               html|
|  6|                css|
|  6|               css3|
|  6|internet-explorer-7|
|  8|                 c#|
|  8|    code-generation|
|  8|                 j#|
|  8|           visualj#|
|  9|                 c#|
|  9|               .net|
|  9|           datetime|
| 11|                 c#|
| 11|           datetime|
| 11|           datediff|
+---+-------------------+
only showing top 20 rows



In [30]:
data = questions_df.join(questions_tags_df, how="inner", on=["Id"]).withColumn("IsWeekend", date_format("CreationDate", 'EEE').isin(["Sat", "Sun"]).cast("int"))

In [31]:
data.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- ClosedDate: string (nullable = true)
 |-- DeletionDate: string (nullable = true)
 |-- Score: integer (nullable = true)
 |-- OwnerUserId: string (nullable = true)
 |-- AnswerCount: string (nullable = true)
 |-- Tag: string (nullable = true)
 |-- IsWeekend: integer (nullable = true)



In [25]:
most_used_tags = data.select("Tag").groupby("Tag").agg(count("*").alias("Count")).orderBy(desc("Count")).limit(10)
most_used_tags.show()

+----------+-------+
|       Tag|  Count|
+----------+-------+
|javascript|1649631|
|      java|1563820|
|       php|1365600|
|        c#|1321027|
|   android|1288338|
|    jquery|1011324|
|    python| 854172|
|      html| 776512|
|       ios| 635227|
|       c++| 627341|
+----------+-------+



In [33]:
most_used_tags_weekdays = data.select("Tag").filter(data.IsWeekend == 0).groupby("Tag").agg(count("*").alias("Count")).orderBy(desc("Count")).limit(3)
most_used_tags_weekdays.show()

+----------+-------+
|       Tag|  Count|
+----------+-------+
|javascript|1343393|
|      java|1261313|
|        c#|1101023|
+----------+-------+



In [34]:
most_used_tags_weekend = data.select("Tag").filter(data.IsWeekend == 1).groupby("Tag").agg(count("*").alias("Count")).orderBy(desc("Count")).limit(3)
most_used_tags_weekend.show()

+----------+------+
|       Tag| Count|
+----------+------+
|javascript|306238|
|      java|302507|
|       php|277940|
+----------+------+

